In [1]:
import gymnasium as gym
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import jax
import jax.numpy as jnp

import numpy as np

# from irbfn_mpc.explicit_planner import ExplicitPlanner
import irbfn_mpc.explicit_planner as ie
import irbfn_mpc.nonlinear_dmpc as nd

In [2]:
from IPython.display import HTML, display
%matplotlib inline

In [3]:
env = gym.make(
    "f1tenth_gym:f1tenth-v0",
    config={
        "map": "L",
        "observation_config": {
            "type": "features",
            "features": [
                "pose_x",
                "pose_y",
                "delta",
                "linear_vel_x",
                "linear_vel_y",
                "pose_theta",
                "ang_vel_z",
                "beta",
            ],
        },
        "num_agents": 1,
        "control_input": "accl",
    },
    render_mode="rgb_array",
)
env = gym.wrappers.RecordVideo(env, "video_explicit_nmpc_frenet")

/usr/local/lib/python3.12/dist-packages/f1tenth_gym/envs/action.py:155: UserWarning: Only one control mode specified, using accl for longitudinal control and defaulting to steering speed for steering
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/irbfn/scripts/video_explicit_nmpc_frenet folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [4]:
track = env.unwrapped.track

In [5]:
import chex
chex.clear_trace_counter()

In [10]:
from importlib import reload
reload(ie)

<module 'irbfn_mpc.explicit_planner' from '/home/irbfn/src/irbfn_mpc/explicit_planner.py'>

In [11]:
planner = ie.ExplicitFrenetPlanner(track=track)

In [12]:
env.unwrapped.clear_render_callback()
env.unwrapped.add_render_callback(planner.render_waypoints)
env.unwrapped.add_render_callback(planner.render_local_plan)
env.unwrapped.add_render_callback(planner.render_mpc_sol)

In [21]:
poses = np.array(
    [
        [
            env.unwrapped.track.raceline.xs[-280],
            env.unwrapped.track.raceline.ys[-280],
            env.unwrapped.track.raceline.yaws[-280],
        ]
    ]
)
obs, info = env.reset(options={"poses": poses})
done = False

step = 0

while not done and step <= 4000:
    current_state = obs["agent_0"]
    if current_state["linear_vel_x"] < 1.0:
        accl = 9.0
        steerv = 0.0
    else:
        accl, steerv = planner.plan(current_state)
    # accl, steerv = planner2.plan(current_state)
    action = env.action_space.sample()
    action[0] = [steerv, accl]
    # action[0] = [steerv[0], accl[0]]
    obs, step_reward, done, truncated, info = env.step(action)
    step += 1
    # print(current_state)
    # print(f"accl {accl} steerv {steerv}")

ey lookup: -0.0007423310694079754
delta lookup: 0.0
vx_car lookup: 1.0800000429153442
vy_car lookup: 0.0
vx_goal lookup: 4.0
wz lookup: 0.0
epsi lookup: -0.022678589008529926
curv lookup: 0.069691002368927
kdtree lookup distance 0.0885071277281764
kdtree lookup inds 2205770
looked up input: [0.  0.  1.  0.  4.  0.  0.  0.1]
accl seq:   [9.51000008e+00 9.50999945e+00 4.74021201e+00 1.98157767e+00
 1.55236131e-09]
steerv seq: [-3.35791080e-02  3.00105009e-01  1.28708811e-01  2.33416718e-02
 -6.15636001e-11]
ey lookup: -0.00100726636851338
delta lookup: 0.0
vx_car lookup: 1.1750999689102173
vy_car lookup: 0.0
vx_goal lookup: 4.0
wz lookup: 0.0
epsi lookup: -0.02448869807426781
curv lookup: 0.069691002368927
kdtree lookup distance 0.1793860233370182
kdtree lookup inds 2205770
looked up input: [0.  0.  1.  0.  4.  0.  0.  0.1]
accl seq:   [9.51000008e+00 9.50999945e+00 4.74021201e+00 1.98157767e+00
 1.55236131e-09]
steerv seq: [-3.35791080e-02  3.00105009e-01  1.28708811e-01  2.33416718e-02

In [22]:
env.close()

Moviepy - Building video /home/irbfn/scripts/video_explicit_nmpc_frenet/rl-video-episode-1.mp4.
Moviepy - Writing video /home/irbfn/scripts/video_explicit_nmpc_frenet/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /home/irbfn/scripts/video_explicit_nmpc_frenet/rl-video-episode-1.mp4


In [23]:
import glob
import io
import base64

for video_file in glob.glob("video_explicit_nmpc_frenet/*.mp4"):
    video = io.open(video_file, "rb").read()
    encoded = base64.b64encode(video).decode("ascii")
    display(
        HTML(
            f"""<video width="800" height="auto" controls>
                <source src="data:video/mp4;base64,{encoded}" type="video/mp4" />
            </video>"""
        )
    )